In [1]:
import numpy as np 
import torch
import torch.nn as nn
from utility_funcs import get_train_labels_test
from scipy import stats
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from sklearn.preprocessing import StandardScaler

In [2]:
!nvidia-smi

Wed Apr 13 21:15:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8     4W /  N/A |      0MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data_path = '../Data/'
train = pd.read_csv(data_path + "train.csv")
labels = pd.read_csv(data_path + "train_labels.csv")
test = pd.read_csv(data_path + "test.csv")

In [4]:
train.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [5]:
labels.head()

,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [6]:
test.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,25968,684,0,2.427357,19.639706,1.0,-1.466372,-1.289973,-4.207928,2.486339,-2.493893,8.0,-1.123555,-1.673048,10.980453,0.419011
1,25968,684,1,-4.950541,-21.747899,1.0,0.983186,-0.569053,1.845924,-3.887978,1.727481,-2.9,0.395231,-0.882233,-1.871399,-0.008525
2,25968,684,2,1.136012,-10.756303,1.0,1.016814,0.964157,2.454749,0.312386,1.154198,-5.6,1.114162,1.525273,-11.584362,0.139812
3,25968,684,3,0.806028,6.504202,1.0,-0.179646,0.969221,-1.035153,-0.457195,0.254962,-2.7,-0.588873,0.608761,-4.241770,-0.462916
4,25968,684,4,1.288253,5.552521,1.0,-0.493805,-1.036124,-1.126402,2.008197,-0.730534,0.0,0.899566,-1.259615,-0.472222,-0.121483


In [7]:
train.describe().round(2)

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
count,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.00,1558080.0,1558080.00,1558080.00,1558080.00,1558080.00
mean,12983.50,331.63,29.50,0.00,-0.00,-0.22,-0.00,-0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00,-0.00,-0.01
std,7496.32,195.83,17.32,2.66,4.40,2.30,3.93,1.68,1.59,3.35,3.24,4.5,2.59,1.92,4.53,39.12
min,0.00,0.00,0.00,-375.06,-434.60,-31.66,-408.38,-23.63,-74.98,-470.50,-407.01,-536.1,-270.35,-43.41,-427.06,-612.55
25%,6491.75,161.75,14.75,-0.50,-0.48,-0.65,-0.49,-0.47,-0.48,-0.49,-0.50,-0.5,-0.52,-0.48,-0.48,-0.58
50%,12983.50,335.00,29.50,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00,-0.00,0.00,0.00
75%,19475.25,501.00,44.25,0.48,0.49,0.33,0.49,0.47,0.51,0.49,0.48,0.5,0.51,0.48,0.49,0.57
max,25967.00,671.00,59.00,335.82,449.59,1.67,436.65,24.87,77.92,442.50,331.25,630.1,367.98,41.87,448.02,630.51


### Data prep

In [8]:
train = train.reset_index(drop = True).merge(labels, on = 'sequence')

In [9]:
train["sequence_in_file"] = train['sequence'] - train['sequence'].min()
test["sequence_in_file"] = test['sequence'] - test['sequence'].min()

ctd = ['sequence', 'subject', 'step']
train.drop(ctd, axis =1, inplace = True)
test.drop(ctd, axis =1, inplace = True)

In [10]:
test.columns, train.columns

(Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
        'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
        'sensor_10', 'sensor_11', 'sensor_12', 'sequence_in_file'],
       dtype='object'),
 Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
        'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
        'sensor_10', 'sensor_11', 'sensor_12', 'state', 'sequence_in_file'],
       dtype='object'))

In [11]:
s=StandardScaler()
X_train = train.drop('state', axis =1)
X_train = s.fit_transform(X_train)
y_train = train['state'].values
X_test = s.transform(test)

In [ ]:
class DatasetGenerator(Dataset):
    """ inherit from the dataset class, pytorch needs len and get item wh"""


    def __init__(self,time_series, labels = None):
        super().__init__()
        self.time_series = time_series
        self.labels = labels
        self.cols = [col for col in self.time_series if 'sensor' in col]

    def __len__(self):
        if not self.labels:
            return len(self.time_series) // 60 
        return len(self.labels)

    def __getitem__(self, index):
        if not self.labels:
            label_tensor = np.nan
        else: 
            label = self.labels[self.labels.sequence_local == index].state.iloc[0]
            label_tensor = torch.tensor(label, dtype=torch.long)

        ts_df = self.time_series[self.time_series.sequence_local == index][self.cols].to_numpy()
        sensor_array = torch.tensor(ts_df, dtype=torch.float32)

        return sensor_array, label_tensor

14

In [23]:
class LSTM(nn.Module):
    def __init__(self,
                seq_length = 60,
                input_dim = X_train.shape[1], 
                lstm_dim = 1024,
                num_layers = 2, # i.e 2 lstm models
                dropout = 0.1,
                classes = 1):


        super().__init__()

        self.LSTM0 = nn.LSTM(input_dim,lstm_dim,num_layers,batch_first = True, dropout = dropout, bidirectional = True)
        self.LSTM1 = nn.LSTM(2*lstm_dim, lstm_dim, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
        self.LSTM2 = nn.LSTM(2* lstm_dim, lstm_dim, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
        self.LSTM3 = nn.LSTM(2*lstm_dim, lstm_dim, num_layers, batch_first = True, dropout = dropout, bidirectional = True)

        self.logits = nn.Sequential(
            nn.ReLU(),
            nn.Linear(lstm_dim * seq_length * 2, classes),
        )

    def forward(self, x):
        out, _ = self.LSTM0(x)
        out, _ = self.LSTM1(out)
        out, _ = self.LSTM2(out)
        out, _ = self.LSTM3(out)
        out = out.reshape(out.shape[0], -1)
        out = self.logits(out)
        return out